# Deep Learning for Malaria Diagnosis
This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018) and (Jason Brownlee, 2019). Acknowledge to NIH and Bangalor Hospital who make available this malaria dataset.

Malaria is an infectuous disease caused by parasites that are transmitted to people through the bites of infected female Anopheles mosquitoes.

The Malaria burden with some key figures:
<font color='red'>
* More than 219 million cases
* Over 430 000 deaths in 2017 (Mostly: children & pregnants)
* 80% in 15 countries of Africa & India
  </font>

![MalariaBurd](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaBurden.png?raw=1)

The malaria diagnosis is performed using blood test:
* Collect patient blood smear
* Microscopic visualisation of the parasit

![MalariaDiag](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaDiag.png?raw=1)
  
Main issues related to traditional diagnosis:
<font color='#ed7d31'>
* resource-constrained regions
* time needed and delays
* diagnosis accuracy and cost
</font>

The objective of this notebook is to apply modern deep learning techniques to perform medical image analysis for malaria diagnosis.

*This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018), (Adrian Rosebrock, 2018) and (Jason Brownlee, 2019)*

## Configuration

In [ ]:
#Mount the local drive project_forder
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/cell_images"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Parasitized  Uninfected


In [ ]:
# Use GPU: Please check if the outpout is '/device:GPU:0'
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

2.19.0


'/device:GPU:0'

## Populating namespaces

In [ ]:
# Importing basic libraries
import os
import random
import shutil
from matplotlib import pyplot
from matplotlib.image import imread
%matplotlib inline

# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D as Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [ ]:
# Define the useful paths for data accessibility
ai_project = '.' #"/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
cell_images_dir = os.path.join(ai_project,'cell_images')
training_path = os.path.join(ai_project,'train')
testing_path = os.path.join(ai_project,'test')

## Prepare DataSet

### *Download* DataSet

In [ ]:
# Download the data in the allocated google cloud-server. If already down, turn downloadData=False
downloadData = True
if downloadData == True:
  indrive = False
  if indrive == True:
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip -P "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
    !unzip "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/cell_images.zip" -d "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/"
    !ls "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
  else: #incloud google server
    !rm -rf cell_images.*
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
    !unzip cell_images.zip >/dev/null 2>&1
    !ls

--2025-10-06 05:21:18--  https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
Resolving data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)... 3.163.189.81, 3.163.189.83, 3.163.189.96, ...
Connecting to data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)|3.163.189.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M  25.6MB/s    in 4.2s    

2025-10-06 05:21:22 (80.7 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]

cell_images  cell_images.zip  drive  sample_data


## Baseline CNN Model
Define a basic ConvNet defined with ConvLayer: Conv2D => MaxPooling2D followed by Flatten => Dense => Dense(output)

![ConvNet](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/ConvNet.png?raw=1)


In [ ]:
# =============================================================================
# ADVANCED CNN MODEL FOR MALARIA CLASSIFICATION - OPTIMIZED VERSION
# 10 Epochs for Faster Training
# =============================================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                      Dropout, BatchNormalization,
                                      GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau,
                                        ModelCheckpoint, Callback)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_curve, auc, precision_recall_curve,
                             accuracy_score, precision_score,
                             recall_score, f1_score)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
# =============================================================================
# DETAILED CHECK: What's inside cell_images folder?
# =============================================================================
print("=" * 70)
print("DETAILED CHECK: Exploring 'cell_images' folder")
print("=" * 70)

cell_images_path = 'cell_images'

if os.path.exists(cell_images_path):
    print(f"\n Contents of '{cell_images_path}/':")

    for item in os.listdir(cell_images_path):
        item_path = os.path.join(cell_images_path, item)

        if os.path.isdir(item_path):
            print(f"\n    {item}/")

            # Check what's inside this subfolder
            sub_items = os.listdir(item_path)
            print(f"      Contains {len(sub_items)} items")

            # Show first few items
            for sub_item in sub_items[:5]:
                sub_path = os.path.join(item_path, sub_item)
                if os.path.isdir(sub_path):
                    num_files = len(os.listdir(sub_path))
                    print(f"       {sub_item}/ ({num_files} files)")
                else:
                    print(f"       {sub_item}")

            if len(sub_items) > 5:
                print(f"      ... and {len(sub_items) - 5} more items")
        else:
            print(f"    {item}")

print("\n" + "=" * 70)

DETAILED CHECK: Exploring 'cell_images' folder

📁 Contents of 'cell_images/':

   📁 Parasitized/
      Contains 13780 items
      📄 C39P4thinF_original_IMG_20150622_105335_cell_8.png
      📄 C143P104ThinF_IMG_20151005_230100_cell_158.png
      📄 C60P21thinF_IMG_20150803_144629_cell_6.png
      📄 C132P93ThinF_IMG_20151004_152505_cell_113.png
      📄 C126P87ThinF_IMG_20151004_105342_cell_114.png
      ... and 13775 more items

   📁 Uninfected/
      Contains 13780 items
      📄 C143P104ThinF_IMG_20151005_230100_cell_134.png
      📄 C92P53ThinF_IMG_20150821_151722_cell_33.png
      📄 C146P107ThinF_IMG_20151018_140342_cell_147.png
      📄 C62P23N_ThinF_IMG_20150818_133527_cell_74.png
      📄 C74P35_ThinF_IMG_20150815_114401_cell_15.png
      ... and 13775 more items



In [ ]:
# =============================================================================
# SPLIT DATA INTO TRAIN AND TEST SETS
# =============================================================================
import shutil
from sklearn.model_selection import train_test_split

def split_data(source_dir, train_dir, test_dir, test_size=0.2, random_state=42):
    """
    Split data into train and test sets
    """
    # Create directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get class folders
    classes = [d for d in os.listdir(source_dir)
               if os.path.isdir(os.path.join(source_dir, d))]

    print(f"Found classes: {classes}")

    for class_name in classes:
        print(f"\nProcessing class: {class_name}")

        # Create class directories in train and test
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        # Get all images for this class
        class_dir = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_dir)
                 if f.endswith(('.png', '.jpg', '.jpeg'))]

        print(f"  Total images: {len(images)}")

        # Split into train and test
        train_images, test_images = train_test_split(
            images, test_size=test_size, random_state=random_state
        )

        print(f"  Train: {len(train_images)}, Test: {len(test_images)}")

        # Copy files to train directory
        for img in train_images:
            src = os.path.join(class_dir, img)
            dst = os.path.join(train_class_dir, img)
            shutil.copy2(src, dst)

        # Copy files to test directory
        for img in test_images:
            src = os.path.join(class_dir, img)
            dst = os.path.join(test_class_dir, img)
            shutil.copy2(src, dst)

    print("\n✓ Data split completed successfully!")

# Check if train/test already exist
if not os.path.exists('train') or not os.path.exists('test'):
    print("=" * 70)
    print("SPLITTING DATA INTO TRAIN AND TEST SETS")
    print("=" * 70)
    print("This may take a few minutes...\n")

    split_data(
        source_dir='cell_images',
        train_dir='train',
        test_dir='test',
        test_size=0.2,
        random_state=42
    )

    print("\n" + "=" * 70)
    print("VERIFICATION")
    print("=" * 70)

    for split in ['train', 'test']:
        print(f"\n{split.upper()}:")
        for class_name in os.listdir(split):
            class_path = os.path.join(split, class_name)
            if os.path.isdir(class_path):
                num_images = len(os.listdir(class_path))
                print(f"  {class_name}: {num_images} images")
else:
    print("=" * 70)
    print("✓ Train and Test directories already exist")
    print("=" * 70)

    for split in ['train', 'test']:
        print(f"\n{split.upper()}:")
        for class_name in os.listdir(split):
            class_path = os.path.join(split, class_name)
            if os.path.isdir(class_path):
                num_images = len(os.listdir(class_path))
                print(f"  {class_name}: {num_images} images")

SPLITTING DATA INTO TRAIN AND TEST SETS
This may take a few minutes...

Found classes: ['Parasitized', 'Uninfected']

Processing class: Parasitized
  Total images: 13779
  Train: 11023, Test: 2756

Processing class: Uninfected
  Total images: 13779
  Train: 11023, Test: 2756

✓ Data split completed successfully!

VERIFICATION

TRAIN:
  Parasitized: 11023 images
  Uninfected: 11023 images

TEST:
  Parasitized: 2756 images
  Uninfected: 2756 images


In [ ]:
# =============================================================================
# CONFIGURATION - OPTIMIZED FOR SPEED
# =============================================================================
class Config:
    IMG_SIZE = (124, 124)
    BATCH_SIZE = 64  # Increased for faster training
    EPOCHS = 10  # REDUCED FROM 30 TO 10
    INPUT_SHAPE = (124, 124, 3)
    TRAIN_DIR = 'train'
    TEST_DIR = 'test'
    RESULTS_DIR = 'advanced_cnn_results'

config = Config()

# Create results directory
os.makedirs(config.RESULTS_DIR, exist_ok=True)

print(f"\n⚡ OPTIMIZED CONFIGURATION:")
print(f"   Epochs: {config.EPOCHS}")
print(f"   Batch Size: {config.BATCH_SIZE}")
print(f"   Mixed Precision: Enabled")
print(f"   Expected training time: ~5-10 minutes per experiment\n")


⚡ OPTIMIZED CONFIGURATION:
   Epochs: 10
   Batch Size: 64
   Mixed Precision: Enabled
   Expected training time: ~5-10 minutes per experiment



In [ ]:
# =============================================================================
# EVALUATION FUNCTIONS
# =============================================================================
def plot_learning_curves(history, experiment_name):
    """
    Plot training and validation loss and accuracy
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot loss
    axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_title(f'{experiment_name}\nTraining and Validation Loss',
                     fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    # Plot accuracy
    axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[1].set_title(f'{experiment_name}\nTraining and Validation Accuracy',
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{config.RESULTS_DIR}/{experiment_name}_learning_curves.png',
                dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, experiment_name):
    """
    Plot confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Parasitized', 'Uninfected'],
                yticklabels=['Parasitized', 'Uninfected'],
                cbar_kws={'label': 'Count'})
    plt.title(f'{experiment_name}\nConfusion Matrix',
             fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{config.RESULTS_DIR}/{experiment_name}_confusion_matrix.png',
                dpi=300, bbox_inches='tight')
    plt.close()

def plot_roc_curve(y_true, y_pred_proba, experiment_name):
    """
    Plot ROC curve
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2,
            label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--',
            label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'{experiment_name}\nROC Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{config.RESULTS_DIR}/{experiment_name}_roc_curve.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    return roc_auc

def evaluate_model(model, test_generator, experiment_name):
    """
    Comprehensive model evaluation
    """
    # Reset generator
    test_generator.reset()

    # Get predictions
    y_pred_proba = model.predict(test_generator, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    y_true = test_generator.classes

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Plot visualizations
    plot_confusion_matrix(y_true, y_pred, experiment_name)
    roc_auc = plot_roc_curve(y_true, y_pred_proba, experiment_name)

    # Return metrics dictionary
    metrics = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC': roc_auc
    }

    return metrics, y_true, y_pred

In [ ]:
# =============================================================================
# DATA PREPARATION
# =============================================================================
def prepare_data_generators(augmentation_level='moderate'):
    """
    Prepare data generators with different augmentation levels
    """
    # Training data generators with different augmentation levels
    if augmentation_level == 'none':
        train_datagen = ImageDataGenerator(rescale=1./255)
    elif augmentation_level == 'light':
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
    elif augmentation_level == 'moderate':
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )
    else:  # aggressive
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.3,
            height_shift_range=0.3,
            shear_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=[0.7, 1.3],
            fill_mode='nearest'
        )

    # Validation data (no augmentation)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Load data
    train_generator = train_datagen.flow_from_directory(
        config.TRAIN_DIR,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        shuffle=True
    )

    test_generator = test_datagen.flow_from_directory(
        config.TEST_DIR,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    return train_generator, test_generator

In [ ]:
# =============================================================================
# MODEL ARCHITECTURES
# =============================================================================
def create_advanced_cnn_v1(dropout_rate=0.5, l2_reg=0.001):
    """
    Experiment 1-3: Base advanced architecture with varying regularization
    """
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_reg), input_shape=config.INPUT_SHAPE),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(dropout_rate * 0.5),

        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(dropout_rate * 0.5),

        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(dropout_rate * 0.5),

        # Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),

        # Dense layers
        Dense(512, activation='relu', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(256, activation='relu'),
        Dropout(dropout_rate * 0.6),
        Dense(1, activation='sigmoid', dtype='float32')  # Float32 for stability
    ])

    return model

def create_deep_cnn(num_blocks=5):
    """
    Experiment 4-5: Deeper architecture
    """
    model = Sequential()

    filters = [32, 64, 128, 256, 512]

    for i in range(num_blocks):
        if i == 0:
            model.add(Conv2D(filters[i], (3, 3), activation='relu',
                           padding='same', input_shape=config.INPUT_SHAPE))
        else:
            model.add(Conv2D(filters[i], (3, 3), activation='relu', padding='same'))

        model.add(BatchNormalization())
        model.add(Conv2D(filters[i], (3, 3), activation='relu', padding='same'))
        model.add(BatchNormalization())

        if i < num_blocks - 1:
            model.add(MaxPooling2D(2, 2))

        model.add(Dropout(0.25))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid', dtype='float32'))

    return model

def create_wide_cnn():
    """
    Experiment 6: Wider architecture (more filters per layer)
    """
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same',
               input_shape=config.INPUT_SHAPE),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),

        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid', dtype='float32')
    ])

    return model

In [ ]:
# =============================================================================
# EXPERIMENT RUNNER
# =============================================================================
def run_experiment(experiment_config):
    """
    Run a single experiment with given configuration
    """
    print(f"\n{'='*80}")
    print(f"Running: {experiment_config['name']}")
    print(f"{'='*80}\n")

    start_time = datetime.now()

    # Prepare data
    train_gen, test_gen = prepare_data_generators(
        experiment_config.get('augmentation', 'moderate')
    )

    # Create model
    model = experiment_config['model_fn'](**experiment_config.get('model_params', {}))

    # Compile model
    optimizer = experiment_config.get('optimizer', Adam(learning_rate=0.0001))
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall()]
    )

    # Callbacks - adjusted for 10 epochs
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    # Train model
    history = model.fit(
        train_gen,
        epochs=config.EPOCHS,
        validation_data=test_gen,
        callbacks=callbacks,
        verbose=1
    )

    # Plot learning curves
    plot_learning_curves(history, experiment_config['name'])

    # Evaluate model
    metrics, y_true, y_pred = evaluate_model(model, test_gen, experiment_config['name'])

    # Add training time and parameters
    elapsed_time = (datetime.now() - start_time).total_seconds()
    metrics['Parameters'] = model.count_params()
    metrics['Final_Train_Acc'] = history.history['accuracy'][-1]
    metrics['Final_Val_Acc'] = history.history['val_accuracy'][-1]
    metrics['Training_Time_Sec'] = elapsed_time

    return metrics, model, history

In [ ]:
# =============================================================================
# DEFINE ALL 7 EXPERIMENTS
# =============================================================================
experiments = [
    {
        'name': 'Exp1_Baseline_Moderate_Aug',
        'description': 'Baseline advanced CNN with moderate augmentation',
        'model_fn': create_advanced_cnn_v1,
        'model_params': {'dropout_rate': 0.5, 'l2_reg': 0.001},
        'augmentation': 'moderate',
        'optimizer': Adam(learning_rate=0.0001)
    },
    {
        'name': 'Exp2_High_Dropout',
        'description': 'Increased dropout for better regularization',
        'model_fn': create_advanced_cnn_v1,
        'model_params': {'dropout_rate': 0.7, 'l2_reg': 0.001},
        'augmentation': 'moderate',
        'optimizer': Adam(learning_rate=0.0001)
    },
    {
        'name': 'Exp3_Strong_L2_Reg',
        'description': 'Stronger L2 regularization',
        'model_fn': create_advanced_cnn_v1,
        'model_params': {'dropout_rate': 0.5, 'l2_reg': 0.01},
        'augmentation': 'moderate',
        'optimizer': Adam(learning_rate=0.0001)
    },
    {
        'name': 'Exp4_Deeper_Network',
        'description': 'Deeper architecture with 5 convolutional blocks',
        'model_fn': create_deep_cnn,
        'model_params': {'num_blocks': 5},
        'augmentation': 'moderate',
        'optimizer': Adam(learning_rate=0.00005)
    },
    {
        'name': 'Exp5_Aggressive_Augmentation',
        'description': 'Aggressive data augmentation',
        'model_fn': create_advanced_cnn_v1,
        'model_params': {'dropout_rate': 0.5, 'l2_reg': 0.001},
        'augmentation': 'aggressive',
        'optimizer': Adam(learning_rate=0.0001)
    },
    {
        'name': 'Exp6_Wide_Network',
        'description': 'Wider network with more filters',
        'model_fn': create_wide_cnn,
        'model_params': {},
        'augmentation': 'moderate',
        'optimizer': Adam(learning_rate=0.00005)
    },
    {
        'name': 'Exp7_SGD_Optimizer',
        'description': 'Using SGD with momentum instead of Adam',
        'model_fn': create_advanced_cnn_v1,
        'model_params': {'dropout_rate': 0.5, 'l2_reg': 0.001},
        'augmentation': 'moderate',
        'optimizer': SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    }
]

In [23]:
# =============================================================================
# RUN ALL EXPERIMENTS
# =============================================================================
all_results = []
total_start_time = datetime.now()

for i, exp in enumerate(experiments, 1):
    try:
        print(f"\n⚡ Progress: {i}/{len(experiments)} experiments")
        metrics, model, history = run_experiment(exp)

        result = {
            'Experiment': exp['name'],
            'Description': exp['description'],
            **metrics
        }
        all_results.append(result)

        # Save model
        model.save(f"{config.RESULTS_DIR}/{exp['name']}_model.keras")

        print(f"\n✓ {exp['name']} completed successfully!")
        print(f"  Accuracy: {metrics['Accuracy']:.4f}")
        print(f"  F1-Score: {metrics['F1-Score']:.4f}")
        print(f"  AUC: {metrics['AUC']:.4f}")
        print(f"  Training Time: {metrics['Training_Time_Sec']:.1f} seconds\n")

    except Exception as e:
        print(f"\n✗ Error in {exp['name']}: {str(e)}\n")
        continue

total_elapsed = (datetime.now() - total_start_time).total_seconds()


⚡ Progress: 1/7 experiments

Running: Exp1_Baseline_Moderate_Aug

Found 22046 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 171s 391ms/step - accuracy: 0.6941 - loss: 1.1214 - precision: 0.6844 - recall: 0.7212 - val_accuracy: 0.5000 - val_loss: 2.0659 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 315ms/step - accuracy: 0.8756 - loss: 0.7959 - precision: 0.8464 - recall: 0.9201 - val_accuracy: 0.6573 - val_loss: 1.4984 - val_precision: 0.5934 - val_recall: 0.9996 - learning_rate: 1.0000e-04
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 314ms/step - accuracy: 0.9038 - loss: 0.7159 - precision: 0.8770 - recall: 0.9399 - val_accuracy: 0.8911 - val_loss: 0.7138 - val_precision: 0.8241 - val_recall: 0.9946 - learning_rate: 1.0000e-04
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 108s 313ms/step - accuracy: 0.9168 - loss: 0.6691 - precision: 0.8899 - recall: 0.9515 - val_accuracy: 0.9340 - val_loss: 0.6057 - val_precision: 0.8929 - val_recall: 0.9862 - learning_rate: 1.0000e-04
Epoch 5/10
345/345 ━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 135s 349ms/step - accuracy: 0.6058 - loss: 1.3971 - precision_1: 0.6194 - recall_1: 0.5446 - val_accuracy: 0.5000 - val_loss: 2.8253 - val_precision_1: 0.5000 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 316ms/step - accuracy: 0.8217 - loss: 0.9293 - precision_1: 0.7925 - recall_1: 0.8715 - val_accuracy: 0.7286 - val_loss: 1.3645 - val_precision_1: 0.6499 - val_recall_1: 0.9909 - learning_rate: 1.0000e-04
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 113s 327ms/step - accuracy: 0.8677 - loss: 0.8186 - precision_1: 0.8358 - recall_1: 0.9181 - val_accuracy: 0.8028 - val_loss: 1.0514 - val_precision_1: 0.7185 - val_recall_1: 0.9956 - learning_rate: 1.0000e-04
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 316ms/step - accuracy: 0.8870 - loss: 0.7683 - precision_1: 0.8562 - recall_1: 0.9355 - val_accuracy: 0.8443 - val_loss: 0.8698 - val_precision_1: 0.7639 - val_recall_1: 0.9967 - learning_rate: 1.0000e-04


...

Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 154s 403ms/step - accuracy: 0.6181 - loss: 1.2292 - precision_4: 0.6145 - recall_4: 0.6364 - val_accuracy: 0.5000 - val_loss: 2.4696 - val_precision_4: 0.5000 - val_recall_4: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 121s 352ms/step - accuracy: 0.8241 - loss: 0.8897 - precision_4: 0.7827 - recall_4: 0.8945 - val_accuracy: 0.6972 - val_loss: 1.2157 - val_precision_4: 0.6229 - val_recall_4: 0.9996 - learning_rate: 1.0000e-04
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 123s 357ms/step - accuracy: 0.8587 - loss: 0.8095 - precision_4: 0.8160 - recall_4: 0.9256 - val_accuracy: 0.8864 - val_loss: 0.7454 - val_precision_4: 0.8173 - val_recall_4: 0.9953 - learning_rate: 1.0000e-04
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 122s 354ms/step - accuracy: 0.8771 - loss: 0.7547 - precision_4: 0.8370 - recall_4: 0.9376 - val_accuracy: 0.9258 - val_loss: 0.6211 - val_precision_4: 0.8779 - val_recall_4: 0.9891 - learning_rate: 1.0000e-04


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 202s 426ms/step - accuracy: 0.6253 - loss: 0.7398 - precision_5: 0.6233 - recall_5: 0.6184 - val_accuracy: 0.5000 - val_loss: 2.6595 - val_precision_5: 0.5000 - val_recall_5: 1.0000 - learning_rate: 5.0000e-05
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 114s 330ms/step - accuracy: 0.8455 - loss: 0.3842 - precision_5: 0.8187 - recall_5: 0.8867 - val_accuracy: 0.6624 - val_loss: 1.1406 - val_precision_5: 0.5983 - val_recall_5: 0.9884 - learning_rate: 5.0000e-05
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 114s 332ms/step - accuracy: 0.8878 - loss: 0.3060 - precision_5: 0.8555 - recall_5: 0.9325 - val_accuracy: 0.8476 - val_loss: 0.5133 - val_precision_5: 0.7677 - val_recall_5: 0.9967 - learning_rate: 5.0000e-05
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 113s 328ms/step - accuracy: 0.8985 - loss: 0.2793 - precision_5: 0.8682 - recall_5: 0.9392 - val_accuracy: 0.8186 - val_loss: 0.5353 - val_precision_5: 0.7345 - val_recall_5: 0.9978 - learning_rate: 5.0000e-05


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 133s 350ms/step - accuracy: 0.8070 - loss: 0.9172 - precision_6: 0.7814 - recall_6: 0.8449 - val_accuracy: 0.6528 - val_loss: 1.0241 - val_precision_6: 0.5903 - val_recall_6: 0.9982 - learning_rate: 0.0100
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 112s 324ms/step - accuracy: 0.9189 - loss: 0.6563 - precision_6: 0.8935 - recall_6: 0.9532 - val_accuracy: 0.9338 - val_loss: 0.5502 - val_precision_6: 0.8960 - val_recall_6: 0.9815 - learning_rate: 0.0100
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 315ms/step - accuracy: 0.9285 - loss: 0.5707 - precision_6: 0.9062 - recall_6: 0.9568 - val_accuracy: 0.9260 - val_loss: 0.5150 - val_precision_6: 0.8765 - val_recall_6: 0.9917 - learning_rate: 0.0100
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 109s 315ms/step - accuracy: 0.9279 - loss: 0.5264 - precision_6: 0.9058 - recall_6: 0.9558 - val_accuracy: 0.9267 - val_loss: 0.4887 - val_precision_6: 0.8791 - val_recall_6: 0.9895 - learning_rate: 0.0100
Epoch 5/10
345/3

In [24]:
# =============================================================================
# GENERATE RESULTS SUMMARY TABLE
# =============================================================================
results_df = pd.DataFrame(all_results)

# Reorder columns
column_order = ['Experiment', 'Description', 'Accuracy', 'Precision',
                'Recall', 'F1-Score', 'AUC', 'Parameters',
                'Final_Train_Acc', 'Final_Val_Acc', 'Training_Time_Sec']
results_df = results_df[column_order]

# Format numeric columns
numeric_cols = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC',
                'Final_Train_Acc', 'Final_Val_Acc']
for col in numeric_cols:
    results_df[col] = results_df[col].apply(lambda x: f"{x:.4f}")

# Format training time
results_df['Training_Time_Sec'] = results_df['Training_Time_Sec'].apply(
    lambda x: f"{x:.1f}s"
)

# Save to CSV
results_df.to_csv(f'{config.RESULTS_DIR}/experiments_summary.csv', index=False)

print("\n" + "="*80)
print("ALL EXPERIMENTS COMPLETED!")
print("="*80)
print(f"\n⏱️  Total Training Time: {total_elapsed/60:.1f} minutes")
print(f"⚡ Average Time per Experiment: {total_elapsed/len(experiments):.1f} seconds")
print("\nResults Summary Table:")
print(results_df.to_string(index=False))
print(f"\nAll results saved to: {config.RESULTS_DIR}/")
print(f"  - Learning curves for each experiment")
print(f"  - Confusion matrices")
print(f"  - ROC curves")
print(f"  - Summary CSV table")
print(f"  - Trained models (.keras format)")
print("\n" + "="*80)


ALL EXPERIMENTS COMPLETED!

⏱️  Total Training Time: 138.0 minutes
⚡ Average Time per Experiment: 1182.6 seconds

Results Summary Table:
                  Experiment                                      Description Accuracy Precision Recall F1-Score    AUC  Parameters Final_Train_Acc Final_Val_Acc Training_Time_Sec
  Exp1_Baseline_Moderate_Aug Baseline advanced CNN with moderate augmentation   0.9512    0.9217 0.9862   0.9528 0.9884      849313          0.9335        0.9512           1161.5s
           Exp2_High_Dropout      Increased dropout for better regularization   0.9147    0.8594 0.9917   0.9208 0.9888      849313          0.9238        0.9147           1130.0s
          Exp3_Strong_L2_Reg                       Stronger L2 regularization   0.9448    0.9070 0.9913   0.9473 0.9895      849313          0.9340        0.9448           1138.0s
         Exp4_Deeper_Network  Deeper architecture with 5 convolutional blocks   0.9243    0.8762 0.9884   0.9289 0.9865     5116449          0